In [1]:
import tensorflow as tf

import numpy as np

import os

import pickle

In [2]:
save_path = "/home/ist_davidcalhas/eeg_to_fmri/na_models/"
dim1_path = "/home/ist_davidcalhas/eeg_to_fmri/na_models_64/uni_wit/"
dim2_path = "/home/ist_davidcalhas/eeg_to_fmri/na_models_134/uni_wit/"
save_spec=True
save=False

In [3]:
models1 = {"1": [], "2": [], "3": []}

"""

        NA_specification - tuple - (list1, list2, bool, tuple1, tuple2)
                                    * list1 - kernel sizes
                                    * list2 - stride sizes
                                    * bool - maxpool
                                    * tuple1 - kernel size of maxpool
                                    * tuple2 - stride size of maxpool
"""                     
for na in (os.listdir(dim1_path)):
    model1 = tf.keras.models.load_model(dim1_path+na)
    
    kernel_sizes = []
    stride_sizes = []
    
    for layer in model1.layers:
        if("conv" in layer.name):
            kernel_sizes += [layer.kernel_size]
            stride_sizes += [layer.strides]
    
    maxpool = True
    max_kernel = (2,)
    max_stride = (1,)
    
    models1[str(int(len(model1.layers)/2))] += [(kernel_sizes, stride_sizes, maxpool, max_kernel, max_stride)]

In [4]:
models2 = {"1": [], "2": [], "3": []}
                 
for na in (os.listdir(dim2_path)):
    model2 = tf.keras.models.load_model(dim2_path+na)
    
    kernel_sizes = []
    stride_sizes = []
    
    for layer in model2.layers:
        if("conv" in layer.name):
            kernel_sizes += [layer.kernel_size]
            stride_sizes += [layer.strides]
    
    maxpool = True
    max_kernel = (2,)
    max_stride = (1,)
    
    models2[str(int(len(model2.layers)/2))] += [(kernel_sizes, stride_sizes, maxpool, max_kernel, max_stride)]

In [5]:
print(len(models1["1"]))
print(len(models1["2"]))
print(len(models1["3"]))

1
8
21


In [6]:
print(len(models2["1"]))
print(len(models2["2"]))
print(len(models2["3"]))

0
23
7


## Match neural architectures

In [7]:
models = {"1": [], "2": [], "3": []}
K=20

for i in range(K):
    print("Model", i+1)
    
    nlayers = np.random.choice([1,2,3])
    
    if(not sum([len(models1["1"])*len(models2["1"]), len(models1["2"])*len(models2["2"]), len(models1["3"])*len(models2["3"])])):
        break
        
    while(not [len(models1["1"])*len(models2["1"]), len(models1["2"])*len(models2["2"]), len(models1["3"])*len(models2["3"])][nlayers-1]):
        nlayers = np.random.choice([1,2,3])
    
    #random choice
    m1_idx = np.random.choice(list(range(len(models1[str(nlayers)]))))
    m2_idx = np.random.choice(list(range(len(models2[str(nlayers)]))))
    
    naspec1 = models1[str(nlayers)][m1_idx]
    naspec2 = models2[str(nlayers)][m2_idx]
    
    
    # build new network
    kernel_sizes = []
    stride_sizes = []
    
    for l in range(nlayers):
        kernel_size1 = models1[str(nlayers)][m1_idx][0][l][0]
        stride_size1 = models1[str(nlayers)][m1_idx][1][l][0]
        
        kernel_size2 = models2[str(nlayers)][m2_idx][0][l][0]
        stride_size2 = models2[str(nlayers)][m2_idx][1][l][0]
        
        kernel_size3 = 2
        stride_size3 = 1
        
        kernel_sizes += [(kernel_size1,kernel_size2,kernel_size3)]
        stride_sizes += [(stride_size1,stride_size2,stride_size3)]        
        
    maxpool=True
    max_kernel = (models1[str(nlayers)][m1_idx][3][0], models2[str(nlayers)][m2_idx][3][0],1)
    max_stride = (models1[str(nlayers)][m1_idx][4][0], models2[str(nlayers)][m2_idx][4][0],1)
    
    if(save_spec):
        with open(save_path+"na_specification_"+str(i+1), "wb") as f:
            pickle.dump((kernel_sizes, stride_sizes, maxpool, max_kernel, max_stride), f)
    models[str(nlayers)] += [(kernel_sizes, stride_sizes, maxpool, max_kernel, max_stride)]
        
    del models1[str(nlayers)][m1_idx]
    del models2[str(nlayers)][m2_idx]

Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
Model 7
Model 8
Model 9
Model 10
Model 11
Model 12
Model 13
Model 14
Model 15
Model 16


### Verify integrity of models and save to path of architectures

In [8]:
input_shape = (64,134,10)
output_shape = (7,7,7)


m_idx = 0
for nlayers_models in models.values():
    for model in nlayers_models:
        
        m = tf.keras.Sequential()
        
        for layer in range(len(model[0])):
            m.add(tf.keras.layers.Conv3D(1, model[0][layer], strides=model[1][layer]))
            if(model[2]):
                m.add(tf.keras.layers.MaxPool3D(model[3], strides=model[4]))
        
        m.build(input_shape=(None,)+input_shape+(1,))
        
        if(save):
            tf.keras.models.save_model(m, save_path+"architecture_"+str(m_idx+1), save_format="tf")
        
        m_idx += 1

In [9]:
## Save models
for i in range(K):
    try:
        with open(save_path+"na_specification_"+str(i+1), "rb") as f:
            print(pickle.load(f))
    except:
        break


([(11, 86, 2), (17, 20, 2), (2, 7, 2)], [(1, 1, 1), (4, 2, 1), (1, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(7, 37, 2), (7, 7, 2)], [(3, 5, 1), (2, 2, 1)], True, (2, 2, 1), (1, 1, 1))
([(9, 43, 2), (11, 11, 2), (9, 3, 2)], [(1, 2, 1), (1, 2, 1), (5, 2, 1)], True, (2, 2, 1), (1, 1, 1))
([(28, 15, 2), (30, 77, 2)], [(1, 1, 1), (1, 7, 1)], True, (2, 2, 1), (1, 1, 1))
([(7, 19, 2), (20, 23, 2), (23, 16, 2)], [(1, 1, 1), (1, 4, 1), (2, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(6, 29, 2), (21, 33, 2), (16, 11, 2)], [(1, 1, 1), (1, 4, 1), (3, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(32, 47, 2), (4, 15, 2)], [(2, 4, 1), (2, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(9, 16, 2), (5, 2, 2), (6, 81, 2)], [(3, 1, 1), (1, 1, 1), (1, 5, 1)], True, (2, 2, 1), (1, 1, 1))
([(23, 32, 2), (11, 96, 2)], [(1, 1, 1), (5, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(16, 31, 2), (24, 6, 2)], [(1, 8, 1), (4, 1, 1)], True, (2, 2, 1), (1, 1, 1))
([(13, 30, 2), (33, 92, 2)], [(1, 1, 1), (3, 2, 1)], True, (2, 2, 1), (1, 1, 1))
([(4,